### N-gram language models or how to write scientific papers (4 pts)

We shall train our language model on a corpora of [ArXiv](http://arxiv.org/) articles and see if we can generate a new one!

![img](https://media.npr.org/assets/img/2013/12/10/istock-18586699-monkey-computer_brick-16e5064d3378a14e0e4c2da08857efe03c04695e-s800-c85.jpg)

_data by neelshah18 from [here](https://www.kaggle.com/neelshah18/arxivdataset/)_

_Disclaimer: this has nothing to do with actual science. But it's fun, so who cares?!_

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Alternative manual download link: https://yadi.sk/d/_nGyU2IajjR9-w
!wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
!tar -xvzf arxivData.json.tar.gz
data = pd.read_json("./arxivData.json")
data.sample(n=5)

--2021-01-05 12:47:16--  https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.64.18, 2620:100:6025:18::a27d:4512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.64.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/99az9n1b57qkd9j/arxivData.json.tar.gz [following]
--2021-01-05 12:47:16--  https://www.dropbox.com/s/dl/99az9n1b57qkd9j/arxivData.json.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3b428b8f8bc344d03af8d6450c.dl.dropboxusercontent.com/cd/0/get/BGY0-8-1L1nvvSLR_jXG6kZVV-A36GbBsz5wi-WcOOKLD7fXsK2cXqnx_y04cw1BPr5PBCIGYzj3zwiAin4MhDJvhMtDapAAjUbNtUvhu60h7w/file?dl=1# [following]
--2021-01-05 12:47:17--  https://uc3b428b8f8bc344d03af8d6450c.dl.dropboxusercontent.com/cd/0/get/BGY0-8-1L1nvvSLR_jXG6kZVV-A36GbBsz5wi-WcOOKLD7fXsK2cXqnx_y04cw1BPr5PBCIGYzj3zwiAin4MhDJvhMtDapAAjUbN

,author,day,id,link,month,summary,tag,title,year
39488,[{'name': 'Gonzalo Vaca-Castano'}],25,1606.07921v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",6,In this paper we introduce the problem of dete...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",Finding the Topic of a Set of Images,2016
8940,"[{'name': 'Wei Wei'}, {'name': 'Kennth Joseph'...",20,1708.06000v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",8,The Recurrent Chinese Restaurant Process (RCRP...,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",Efficient Online Inference for Infinite Evolut...,2017
9343,"[{'name': 'Chris Hinrichs'}, {'name': 'Vamsi K...",12,1502.03536v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,Multiple hypothesis testing is a significant p...,"[{'term': 'stat.CO', 'scheme': 'http://arxiv.o...",Speeding up Permutation Testing in Neuroimaging,2015
29921,"[{'name': 'Shuangjun Liu'}, {'name': 'Yu Yin'}...",3,1711.01005v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",11,Although human pose estimation for various com...,"[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",In-Bed Pose Estimation: Deep Learning with Sha...,2017
32869,"[{'name': 'Hamid Reza Hassanzadeh'}, {'name': ...",29,1509.08888v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",9,Prediction of survival for cancer patients is ...,"[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",A Semi-Supervised Method for Predicting Cancer...,2015


In [10]:
# assemble lines: concatenate title and description
lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'], axis=1).tolist()

sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

### Tokenization

You know the dril. The data is messy. Go clean the data. Use WordPunctTokenizer or something.


In [11]:
# Task: convert lines (in-place) into strings of space-separated tokens. import & use WordPunctTokenizer
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()
lines = [' '.join(tokenizer.tokenize(l.lower())) for l in lines]

In [13]:
assert sorted(lines, key=len)[0] == \
    'differential contrastive divergence ; this paper has been retracted .'
assert sorted(lines, key=len)[2] == \
    'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .'

### N-Gram Language Model (1point)

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$ 

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_t \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words. 

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

You can also sometimes see such approximation under the name of _n-th order markov assumption_.

The first stage to building such a model is counting all word occurences given N-1 previous words

In [18]:
lines[0]

'dual recurrent attention units for visual question answering ; we propose an architecture for vqa which utilizes recurrent layers to generate visual and textual attention . the memory characteristic of the proposed recurrent attention units offers a rich joint embedding of visual and textual features and enables the model to reason relations between several parts of the image and question . our single model outperforms the first place winner on the vqa 1 . 0 dataset , performs within margin to the current state - of - the - art ensemble model . we also experiment with replacing attention mechanisms in other state - of - the - art models with our implementation and show increased accuracy . in both cases , our recurrent attention mechanism improves performance in tasks requiring sequential or relational reasoning on the vqa dataset .'

In [243]:
lines[0]

'dual recurrent attention units for visual question answering ; we propose an architecture for vqa which utilizes recurrent layers to generate visual and textual attention . the memory characteristic of the proposed recurrent attention units offers a rich joint embedding of visual and textual features and enables the model to reason relations between several parts of the image and question . our single model outperforms the first place winner on the vqa 1 . 0 dataset , performs within margin to the current state - of - the - art ensemble model . we also experiment with replacing attention mechanisms in other state - of - the - art models with our implementation and show increased accuracy . in both cases , our recurrent attention mechanism improves performance in tasks requiring sequential or relational reasoning on the vqa dataset .'

In [191]:
from collections import defaultdict, Counter
n = 2
UNK, EOS = "_UNK_", "_EOS_"

_increment_count = """
first, second, third = {input}
"""

counts = defaultdict(Counter)

for line in lines[0:2]:
    sentence = line.split()
    sentence = n * [UNK] + sentence + [EOS]
    for i in range(len(sentence[n:])):
        counts[(sentence[i], sentence[i+1])][sentence[i+2]] = counts[(sentence[i], sentence[i+1])].get(sentence[i+2], 0) + 1
    #     print(sentence[i], sentence[i+1], word)

In [321]:
n = 4
UNK, EOS = "_UNK_", "_EOS_"



In [325]:
for prefix, word in get_prefix(lines[0:1], 4):
    print(prefix, word)

('_UNK_', '_UNK_', '_UNK_') dual
('_UNK_', '_UNK_', 'dual') recurrent
('_UNK_', 'dual', 'recurrent') attention
('dual', 'recurrent', 'attention') units
('recurrent', 'attention', 'units') for
('attention', 'units', 'for') visual
('units', 'for', 'visual') question
('for', 'visual', 'question') answering
('visual', 'question', 'answering') ;
('question', 'answering', ';') we
('answering', ';', 'we') propose
(';', 'we', 'propose') an
('we', 'propose', 'an') architecture
('propose', 'an', 'architecture') for
('an', 'architecture', 'for') vqa
('architecture', 'for', 'vqa') which
('for', 'vqa', 'which') utilizes
('vqa', 'which', 'utilizes') recurrent
('which', 'utilizes', 'recurrent') layers
('utilizes', 'recurrent', 'layers') to
('recurrent', 'layers', 'to') generate
('layers', 'to', 'generate') visual
('to', 'generate', 'visual') and
('generate', 'visual', 'and') textual
('visual', 'and', 'textual') attention
('and', 'textual', 'attention') .
('textual', 'attention', '.') the
('attention'

In [193]:
counts

defaultdict(collections.Counter,
            {('_UNK_', '_UNK_'): Counter({'dual': 1, 'sequential': 1}),
             ('_UNK_', 'dual'): Counter({'recurrent': 1}),
             ('dual', 'recurrent'): Counter({'attention': 1}),
             ('recurrent', 'attention'): Counter({'units': 2, 'mechanism': 1}),
             ('attention', 'units'): Counter({'for': 1, 'offers': 1}),
             ('units', 'for'): Counter({'visual': 1}),
             ('for', 'visual'): Counter({'question': 1}),
             ('visual', 'question'): Counter({'answering': 1}),
             ('question', 'answering'): Counter({';': 1}),
             ('answering', ';'): Counter({'we': 1}),
             (';', 'we'): Counter({'propose': 1}),
             ('we', 'propose'): Counter({'an': 1}),
             ('propose', 'an'): Counter({'architecture': 1}),
             ('an', 'architecture'): Counter({'for': 1}),
             ('architecture', 'for'): Counter({'vqa': 1}),
             ('for', 'vqa'): Counter({'which': 1}),

In [332]:
from tqdm import tqdm
from collections import defaultdict, Counter

# special tokens: 
# - unk represents absent tokens, 
# - eos is a special token after the end of sequence

UNK, EOS = "_UNK_", "_EOS_"


# Wrote this helper function
def get_prefix(line, n):
    sentence = line.split()
    sentence.append(EOS)
    for i, word in enumerate(sentence):
        prefix = []
        if i < n - 1:
            for k in range(n - i - 1):
                prefix.append(UNK)
            if k < n - 1:
                for l in range(n - k - 2):
                    prefix.append(sentence[l])
            yield(tuple(prefix), word)
            continue
        for j in range(i+1-n, i):
            prefix.append(sentence[j])

        yield(tuple(prefix), word)

def count_ngrams(lines, n):
    """
    Count how many times each word occured after (n - 1) previous words
    :param lines: an iterable of strings with space-separated tokens
    :returns: a dictionary { tuple(prefix_tokens): {next_token_1: count_1, next_token_2: count_2}}

    When building counts, please consider the following two edge cases
    - if prefix is shorter than (n - 1) tokens, it should be padded with UNK. For n=3,
      empty prefix: "" -> (UNK, UNK)
      short prefix: "the" -> (UNK, the)
      long prefix: "the new approach" -> (new, approach)
    - you should add a special token, EOS, at the end of each sequence
      "... with deep neural networks ." -> (..., with, deep, neural, networks, ., EOS)
      count the probability of this token just like all others.
    """
    counts = defaultdict(Counter)
    # counts[(word1, word2)][word3] = how many times word3 occured after (word1, word2)
    

    for line in lines:
        for prefix, word in get_prefix(line, n):
            counts[prefix][word] = \
            counts[prefix].get(word, 0) + 1

    return counts


In [333]:
# let's test it
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[('_UNK_', '_UNK_')]) == 78
assert dummy_counts['_UNK_', 'a']['note'] == 3
assert dummy_counts['p', '=']['np'] == 2
assert dummy_counts['author', '.']['_EOS_'] == 1

In [353]:
probs = defaultdict(Counter)
for prefix, counts in dummy_counts.items():
    S = sum(counts.values())
    for word in counts.keys():
        print(prefix, word)
        print(dummy_counts[prefix])
        probs[prefix][word] = dummy_counts[prefix][word]/S
    

('_UNK_', '_UNK_') differential
Counter({'a': 13, 'the': 3, 'on': 3, 'using': 2, 'learning': 2, 'automatic': 2, 'why': 2, 'proceedings': 2, 'piecewise': 2, 'differential': 1, 'what': 1, 'p': 1, 'computational': 1, 'weak': 1, 'creating': 1, 'defeasible': 1, 'essence': 1, 'deep': 1, 'statistical': 1, 'complex': 1, 'serious': 1, 'preprocessing': 1, 'liquid': 1, 'mining': 1, 'towards': 1, 'icon': 1, 'recognition': 1, 'glottochronologic': 1, 'utility': 1, 'temporized': 1, 'backpropagation': 1, 'random': 1, 'network': 1, 'glottochronology': 1, 'time': 1, 'convolutional': 1, 'fitness': 1, 'flip': 1, 'autonomous': 1, 'activitynet': 1, 'decision': 1, 'text': 1, 'discrimination': 1, 'are': 1, 'extraction': 1, 'comments': 1, 'resource': 1, 'advances': 1, 'exploration': 1, 'quantified': 1, 'in': 1, 'introduction': 1, 'beyond': 1, 'norm': 1, 'about': 1, 'unary': 1, 'some': 1, 'convex': 1, 'neurocontrol': 1, 'philosophy': 1, 'parallels': 1, 'an': 1, 'calculate': 1, 'group': 1, 'entropy': 1, 'word': 

('ccca', ')') _EOS_
Counter({'_EOS_': 1})
('a', 'machine') -
Counter({'-': 1, 'learning': 1})
('a', 'machine') learning
Counter({'-': 1, 'learning': 1})
('machine', '-') learning
Counter({'learning': 1})
('-', 'learning') framework
Counter({'framework': 1})
('learning', 'framework') for
Counter({'for': 1})
('framework', 'for') design
Counter({'design': 1, 'mapping': 1})
('framework', 'for') mapping
Counter({'design': 1, 'mapping': 1})
('for', 'design') for
Counter({'for': 1})
('design', 'for') manufacturability
Counter({'manufacturability': 1})
('for', 'manufacturability') ;
Counter({';': 1})
('manufacturability', ';') this
Counter({'this': 1})
('a', 'duplicate') submission
Counter({'submission': 1})
('duplicate', 'submission') (
Counter({'(': 1})
('submission', '(') original
Counter({'original': 1})
('(', 'original') is
Counter({'is': 1})
('original', 'is') arxiv
Counter({'arxiv': 1})
('is', 'arxiv') :
Counter({':': 1})
('arxiv', ':') 1612
Counter({'1612': 1, '1306': 1, '0711': 1})
('

('somewhat', 'complicated') .
Counter({'.': 1})
('complicated', '.') _EOS_
Counter({'_EOS_': 1})
('_UNK_', 'word') segmentation
Counter({'segmentation': 1})
('word', 'segmentation') on
Counter({'on': 2})
('segmentation', 'on') micro
Counter({'micro': 2})
('on', 'micro') -
Counter({'-': 2})
('micro', '-') blog
Counter({'blog': 2})
('-', 'blog') texts
Counter({'texts': 2})
('blog', 'texts') with
Counter({'with': 1, '.': 1})
('blog', 'texts') .
Counter({'with': 1, '.': 1})
('texts', 'with') external
Counter({'external': 1})
('with', 'external') lexicon
Counter({'lexicon': 1})
('external', 'lexicon') and
Counter({'and': 1})
('lexicon', 'and') heterogeneous
Counter({'heterogeneous': 1})
('and', 'heterogeneous') data
Counter({'data': 1})
('heterogeneous', 'data') ;
Counter({';': 1})
('data', ';') this
Counter({'this': 1})
('paper', 'describes') our
Counter({'our': 1})
('describes', 'our') system
Counter({'system': 1})
('our', 'system') designed
Counter({'designed': 1})
('system', 'designed')

Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [356]:
class NGramLanguageModel:    
    def __init__(self, lines, n):
        """ 
        Train a simple count-based language model: 
        compute probabilities P(w_t | prefix) given ngram counts
        
        :param n: computes probability of next token given (n - 1) previous words
        :param lines: an iterable of strings with space-separated tokens
        """
        assert n >= 1
        self.n = n
    
        counts = count_ngrams(lines, self.n)
        
        # compute token proabilities given counts
        self.probs = defaultdict(Counter)
        # probs[(word1, word2)][word3] = P(word3 | word1, word2)
        
        # populate self.probs with actual probabilities
        for prefix, counter in counts.items():
            S = sum(counter.values())
            for word, frequency in counter.items():
                self.probs[prefix][word] = frequency/S
            
    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : it's probability} for all tokens with positive probabilities
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [ UNK ] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]
    
    def get_next_token_prob(self, prefix, next_token):
        """
        :param prefix: string with space-separated prefix tokens
        :param next_token: the next token to predict probability for
        :returns: P(next_token|prefix) a single number, 0 <= P <= 1
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)

Let's test it!

In [355]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

p_initial = dummy_lm.get_possible_next_tokens('') # '' -> ['_UNK_', '_UNK_']
assert np.allclose(p_initial['learning'], 0.02)
assert np.allclose(p_initial['a'], 0.13)
assert np.allclose(p_initial.get('meow', 0), 0)
assert np.allclose(sum(p_initial.values()), 1)

p_a = dummy_lm.get_possible_next_tokens('a') # '' -> ['_UNK_', 'a']
assert np.allclose(p_a['machine'], 0.15384615)
assert np.allclose(p_a['note'], 0.23076923)
assert np.allclose(p_a.get('the', 0), 0)
assert np.allclose(sum(p_a.values()), 1)

assert np.allclose(dummy_lm.get_possible_next_tokens('a note')['on'], 1)
assert dummy_lm.get_possible_next_tokens('a machine') == \
    dummy_lm.get_possible_next_tokens("there have always been ghosts in a machine"), \
    "your 3-gram model should only depend on 2 previous words"

Now that you've got a working n-gram language model, let's see what sequences it can generate. But first, let's train it on the whole dataset.

In [357]:
lm = NGramLanguageModel(lines, n=3)

The process of generating sequences is... well, it's sequential. You maintain a list of tokens and iteratively add next token by sampling with probabilities.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Instead of sampling with probabilities, one can also try always taking most likely token, sampling among top-K most likely tokens or sampling with temperature. In the latter case (temperature), one samples from

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$

Where $\tau > 0$ is model temperature. If $\tau << 1$, more likely tokens will be sampled with even higher probability while less likely tokens will vanish.

In [382]:
lm.get_possible_next_tokens(prefix).most_common(1)[0][0]

'learning'

In [372]:
probs

[8.210113217461269e-06,
 0.36552245055459315,
 0.00020525283043653175,
 0.0006650191706143628,
 8.210113217461269e-06,
 8.210113217461269e-06,
 0.0002955640758286057,
 0.00040229554765560223,
 0.006436728762489636,
 7.389101895715143e-05,
 8.210113217461269e-06,
 0.0047290252132576914,
 0.00020525283043653175,
 3.2840452869845076e-05,
 8.210113217461269e-06,
 3.2840452869845076e-05,
 0.0001313618114793803,
 8.210113217461269e-06,
 8.210113217461269e-06,
 8.210113217461269e-06,
 7.389101895715143e-05,
 8.210113217461269e-06,
 8.210113217461269e-06,
 8.210113217461269e-06,
 8.210113217461269e-06,
 8.210113217461269e-06,
 8.210113217461269e-06,
 8.210113217461269e-06,
 8.210113217461269e-06,
 8.210113217461269e-06,
 8.210113217461269e-06,
 8.210113217461269e-06,
 8.210113217461269e-06,
 0.00020525283043653175,
 8.210113217461269e-06,
 3.2840452869845076e-05,
 8.210113217461269e-06,
 8.210113217461269e-06,
 8.210113217461269e-06,
 8.210113217461269e-06,
 8.210113217461269e-06,
 8.210113217

In [374]:
prefix ='a machine'

words, probs
np.random.choice(words, size=1, replace=True, p=probs/np.sum(probs))[0]

'learning'

In [383]:
def get_next_token(lm, prefix, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ (1 / temperature)
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    words = []
    probs = []
    if temperature == 0:
        return lm.get_possible_next_tokens(prefix).most_common(1)[0][0]
    for word, proba in lm.get_possible_next_tokens(prefix).items():
        words.append(word)
        probs.append(proba**(1 / temperature))
    
    return np.random.choice(words, size=1, replace=True, p=probs/np.sum(probs))[0]

In [384]:
from collections import Counter
test_freqs = Counter([get_next_token(lm, 'there have') for _ in range(10000)])
assert 250 < test_freqs['not'] < 450
assert 8500 < test_freqs['been'] < 9500
assert 1 < test_freqs['lately'] < 200

test_freqs = Counter([get_next_token(lm, 'deep', temperature=1.0) for _ in range(10000)])
assert 1500 < test_freqs['learning'] < 3000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.5) for _ in range(10000)])
assert 8000 < test_freqs['learning'] < 9000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.0) for _ in range(10000)])
assert test_freqs['learning'] == 10000

print("Looks nice!")

Looks nice!


Let's have fun with this model

In [385]:
prefix = 'artificial' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

artificial immune system will display the ability of deep learning '. however , the samples of lower dimensions using multi - scale update rule that is independent of the environment , the performance analysis . at the same meaning and grammatical levels . a healthy diet . we tackle in this work , we define a new algorithm to effectively capture the compositional structure of a projection matrix that results are obtained . we achieve an auc ( area under curve of 0 . 503 for macro - economic systems are reported and analyzed on various datasets including surveillance videos wear


In [386]:
prefix = 'bridging the' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break
        
print(prefix)

bridging the gap between the two - step forecast . the proposed method outperforms the state - of - the - art methods . _EOS_


__More in the homework:__ nucleous sampling, top-k sampling, beam search(not for the faint of heart).

### Evaluating language models: perplexity (1point)

Perplexity is a measure of how well does your model approximate true probability distribution behind data. __Smaller perplexity = better model__.

To compute perplexity on one sentence, use:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


On the corpora level, perplexity is a product of probabilities of all tokens in all sentences to the power of 1, divided by __total length of all sentences__ in corpora.

This number can quickly get too small for float32/float64 precision, so we recommend you to first compute log-perplexity (from log-probabilities) and then take the exponent.

In [ ]:
def perplexity(lm, lines, min_logprob=np.log(10 ** -50.)):
    """
    :param lines: a list of strings with space-separated tokens
    :param min_logprob: if log(P(w | ...)) is smaller than min_logprop, set it equal to min_logrob
    :returns: corpora-level perplexity - a single scalar number from the formula above
    
    Note: do not forget to compute P(w_first | empty) and P(eos | full_sequence)
    
    PLEASE USE lm.get_next_token_prob and NOT lm.get_possible_next_tokens
    """
    <YOUR CODE>
    
    return <...>

In [ ]:
lm1 = NGramLanguageModel(dummy_lines, n=1)
lm3 = NGramLanguageModel(dummy_lines, n=3)
lm10 = NGramLanguageModel(dummy_lines, n=10)

ppx1 = perplexity(lm1, dummy_lines)
ppx3 = perplexity(lm3, dummy_lines)
ppx10 = perplexity(lm10, dummy_lines)
ppx_missing = perplexity(lm3, ['the jabberwock , with eyes of flame , '])  # thanks, L. Carrol

print("Perplexities: ppx1=%.3f ppx3=%.3f ppx10=%.3f" % (ppx1, ppx3, ppx10))

assert all(0 < ppx < 500 for ppx in (ppx1, ppx3, ppx10)), "perplexity should be nonnegative and reasonably small"
assert ppx1 > ppx3 > ppx10, "higher N models should overfit and "
assert np.isfinite(ppx_missing) and ppx_missing > 10 ** 6, "missing words should have large but finite perplexity. " \
    " Make sure you use min_logprob right"
assert np.allclose([ppx1, ppx3, ppx10], (318.2132342216302, 1.5199996213739575, 1.1838145037901249))

Now let's measure the actual perplexity: we'll split the data into train and test and score model on test data only.

In [ ]:
from sklearn.model_selection import train_test_split
train_lines, test_lines = train_test_split(lines, test_size=0.25, random_state=42)

for n in (1, 2, 3):
    lm = NGramLanguageModel(n=n, lines=train_lines)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))


In [ ]:
# whoops, it just blew up :)

### LM Smoothing

The problem with our simple language model is that whenever it encounters an n-gram it has never seen before, it assigns it with the probabilitiy of 0. Every time this happens, perplexity explodes.

To battle this issue, there's a technique called __smoothing__. The core idea is to modify counts in a way that prevents probabilities from getting too low. The simplest algorithm here is Additive smoothing (aka [Lapace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)):

$$ P(w_t | prefix) = { Count(prefix, w_t) + \delta \over \sum_{\hat w} (Count(prefix, \hat w) + \delta) } $$

If counts for a given prefix are low, additive smoothing will adjust probabilities to a more uniform distribution. Not that the summation in the denominator goes over _all words in the vocabulary_.

Here's an example code we've implemented for you:

In [ ]:
class LaplaceLanguageModel(NGramLanguageModel): 
    """ this code is an example, no need to change anything """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        counts = count_ngrams(lines, self.n)
        self.vocab = set(token for token_counts in counts.values() for token in token_counts)
        self.probs = defaultdict(Counter)

        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values()) + delta * len(self.vocab)
            self.probs[prefix] = {token: (token_counts[token] + delta) / total_count
                                          for token in token_counts}
    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}
    
    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        

In [ ]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [ ]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

In [ ]:
# optional: try to sample tokens from such a model

### Kneser-Ney smoothing (2 points)

Additive smoothing is simple, reasonably good but definitely not a State of The Art algorithm.


Your final task in this notebook is to implement [Kneser-Ney](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing) smoothing.

It can be computed recurrently, for n>1:

$$P_{kn}(w_t | prefix_{n-1}) = { \max(0, Count(prefix_{n-1}, w_t) - \delta) \over \sum_{\hat w} Count(prefix_{n-1}, \hat w)} + \lambda_{prefix_{n-1}} \cdot P_{kn}(w_t | prefix_{n-2})$$

where
- $prefix_{n-1}$ is a tuple of {n-1} previous tokens
- $lambda_{prefix_{n-1}}$ is a normalization constant chosen so that probabilities add up to 1
- Unigram $P_{kn}(w_t | prefix_{n-2})$ corresponds to Kneser Ney smoothing for {N-1}-gram language model.
- Unigram $P_{kn}(w_t)$ is a special case: how likely it is to see x_t in an unfamiliar context

See lecture slides or wiki for more detailed formulae.

__Your task__ is to
- implement KneserNeyLanguageModel
- test it on 1-3 gram language models
- find optimal (within reason) smoothing delta for 3-gram language model with Kneser-Ney smoothing

In [ ]:
class KneserNeyLanguageModel(NGramLanguageModel): 
    """ A template for Kneser-Ney language model. Default delta may be suboptimal. """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        <YOUR CODE>
        
    def get_possible_next_tokens(self, prefix):
        < YOUR CODE >
        
    def get_next_token_prob(self, prefix, next_token):
        <YOUR CODE>

In [ ]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = KneserNeyLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [ ]:
for n in (1, 2, 3):
    lm = KneserNeyLanguageModel(train_lines, n=n, smoothing=<...>)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))